In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os

In [4]:
mainpath = "D:/Archivos Jordan Rodriguez/Retos/11.SrMD/04.Scraping/"
filename = "md.csv"
fullpath = os.path.join(mainpath,filename)

In [5]:
data = pd.read_csv(fullpath,sep=';')
data.head()

,sitio,url
0,rpp,http://www.rpp.com.pe/2014-09-10-relacion-de-m...
1,rpp,http://www.rpp.com.pe/2014-09-30-mauricio-diez...
2,rpp,https://rpp.pe/famosos/celebridades/mauricio-d...
3,rpp,https://rpp.pe/famosos/chollywood/brad-pizza-f...
4,rpp,https://rpp.pe/famosos/chollywood/leslie-casti...


In [84]:
titulo = []
resumen = []
fecha = []
noticia = []
proxies = {"https": "https://pxsis.sunat.peru:8080"}
for i in range(len(data)):
    url = data["url"][i]
    req = requests.get(url, proxies = proxies, verify = False)
    status_code = req.status_code
    if status_code == 200:
        print(i)
        html = BeautifulSoup(req.text, "html.parser")
        header = html.find('header',class_='story-header')
        titulo.append(header.find('h1',itemprop='headline').getText())
        resumen.append(header.find('div',class_='sumary').getText())
        fecha.append(header.find('time').getText())

        #Identifica el cuerpo del artículo
        contenido = html.find('div',itemprop = 'articleBody')
        texto = contenido.find_all('p')
        index = []
        for i,entrada in enumerate(texto):
            index.append(entrada.getText())
        noticia.append(" ".join(index))
        
        
MD = pd.DataFrame({
    "Título": titulo,
    "Fecha": fecha,
    "Resumen": resumen,
    "Contenido": noticia
    })

MD.to_excel(fullpath + "scra_su.xlsx")

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


0


AttributeError: 'NoneType' object has no attribute 'find'

In [155]:

url = "https://rpp.pe/famosos/chollywood/mauricio-diez-canseco-vuelve-a-la-television-este-sabado-noticia-714285"
req = requests.get(url)

In [7]:
#Para RPP Noticias
#Se extrae el código html de la url solicitada
html = BeautifulSoup(req.text, "html.parser")
#Identifica la cabecera que contiene la información del título, el resumen y la fecha
header = html.find('header',class_='story-header')
titulo = header.find('h1',itemprop='headline').getText()
resumen = header.find('div',class_='sumary').getText()
fecha = header.find('time').getText()

#Identifica el cuerpo del artículo
contenido = html.find('div',itemprop = 'articleBody')
texto = contenido.find_all('p')
index = []
for i,entrada in enumerate(texto):
   index.append(entrada.getText())
noticia = " ".join(index)

#Creamos el data frame de los resultados

MD = pd.DataFrame({
    "Título": titulo,
    "Fecha": fecha,
    "Resumen": resumen,
    "Contenido": noticia
    }, index = [0])
MD


AttributeError: 'NoneType' object has no attribute 'find'

In [3]:
from selenium import webdriver

In [24]:
search_query = input("Ingrese el personaje buscado: ")
search_query = search_query.replace(' ','+') #Cambiar los espacios por + para formato de búsqueda
executable_path = "D:/Cursos/Sunat/11.scraping/chromedriver"
#browser = webdriver.Chrome(executable_path=executable_path)

Ingrese el personaje buscado: Jordan King


In [20]:
browser.get("https://www.google.com/search?q=" + search_query + "&start=0")
matched_elements = browser.find_elements_by_xpath('')
matched_elements[0].click()

InvalidSelectorException: Message: invalid selector: Unable to locate an element with the xpath expression  because of the following error:
SyntaxError: Failed to execute 'evaluate' on 'Document': The string '' is not a valid XPath expression.
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.41.578737 (49da6702b16031c40d63e5618de03a32ff6c197e),platform=Windows NT 10.0.17134 x86_64)


In [60]:
# -*- coding: utf-8 -*- 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
 
def ir_paginas_blancas_web(cadena):
    driver = webdriver.Chrome(executable_path=executable_path)
    #Página a la que queremos acceder
    driver.get("http://www.paginasblancas.pe")
    lista_datos = []
    try:
        #Verificamos si el elemento con ID="nName" ya está cargado, este elemento es la caja de texto donde se hacen las busquedas
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "nName")))
        #Obtenemos la caja de texto de busquedas
        input_nombre = driver.find_element_by_id("nName")
        #Enviamos la cadena que estamos buscando
        input_nombre.send_keys(cadena)
        #Obtenemos el botón que ejecuta la búsqueda
        boton = driver.find_element_by_id("btnSrchName")
        #Damos click al botón
        boton.click()
    except:
        #Mostramos este mensaje en caso de que se presente algún problema
        print("El elemento no está presente")        
    try:
        #Si se encuentran resultados la página los muestra en elementos de nombre "m-results-business"
        #Para ello esperamos que estos elementos se carguen para proceder a consultarlos
        WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "m-results-business")))
    except:
        print('Elementos no encontrados')
    #Obtenemos en una lista los elementos encontrados
    resultados = driver.find_elements_by_class_name("m-results-business")            
    for resultado in resultados:
        #En cada uno de los elementos encontrados buscamos un elemento interno que tiene por nombre info
        datos = resultado.find_element_by_class_name("info")
        #Dentro del elemento interno encontrado buscamos aquellos elementos que son enlaces(etiqueta <a>)
        nombre = datos.find_element_by_tag_name('a')
        #Comprobamos si la cadena que buscamos coincide con la razón social o el nombre mostrado por la página
        if nombre.text.upper().find(cadena)!=-1 or cadena.find(nombre.text.upper())!=-1:
            #Obtenemos la parte de la dirección
            span = datos.find_elements_by_tag_name('span')
            direccion = span[2].text
            partes_ubigeo = span[3].text.split('-')
            distrito = partes_ubigeo[0].strip()
            try:
                departamento = partes_ubigeo[1].strip()
            except:
                departamento = '-'
        #Obtenemos los Datos telefonicos para ello debemos simular el click en el botón que dice "Ver Teléfono"
        boton_telefono = resultado.find_element_by_class_name("m-button--results-business--icon")
        boton_telefono.click()
        #Obtenemos los teléfonos que se muestran como enlaces
        enlaces = resultado.find_elements_by_tag_name('a')
        telefono = enlaces[1].text.replace(' ', '')
        #Finalmente metemos en una lista de listas los datos obtenidos
        lista_datos.append([nombre,nombre.text,direccion,distrito,departamento,telefono])
                 
    driver.close()
    return lista_datos
 
#def main():
    #print (ir_paginas_blancas_web('UNIVERSIDAD DE LIMA'))
         
#main()

In [61]:
ir_paginas_blancas_web('UNIVERSIDAD')

[[<selenium.webdriver.remote.webelement.WebElement (session="c1cb3acb59bb84b044d69e08e735ac93", element="0.8308728190076407-17")>,
  'Universidad Alas Peruanas',
  'Av. San Felipe 1109',
  'Jesús María',
  'Lima',
  'http://www.uap.edu.pe'],
 [<selenium.webdriver.remote.webelement.WebElement (session="c1cb3acb59bb84b044d69e08e735ac93", element="0.8308728190076407-40")>,
  'Universidad Catolica Santo Toribio De Mogrovejo',
  'Avenida Panamericana Norte, 855',
  'Lambayeque',
  '-',
  'http://www.usat.edu.pe'],
 [<selenium.webdriver.remote.webelement.WebElement (session="c1cb3acb59bb84b044d69e08e735ac93", element="0.8308728190076407-62")>,
  'Universidad De Lima',
  'Avenida Javier Prado, 46',
  'Santiago de Surco',
  'Lima',
  'http://www.ulima.edu.pe'],
 [<selenium.webdriver.remote.webelement.WebElement (session="c1cb3acb59bb84b044d69e08e735ac93", element="0.8308728190076407-86")>,
  'Universidad De Lima',
  'Avenida Javier Prado Este - Cdra. 46 s/n - Monterrico',
  'Santiago de Surco'

In [265]:
# -*- coding: utf-8 -*- 
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import os
import time 

#proxies = {"https": "https://pxsis.sunat.peru:8080"}
cadena = '"Mauricio Diez Canseco" site:larepublica.pe'
executable_path = "D:/Archivos Jordan Rodriguez/Retos/11.SrMD/04.Scraping/chromedriver"
num_paginas = 34
#def ir_paginas_blancas_web(cadena):
driver = webdriver.Chrome(executable_path=executable_path)
driver.get("https://www.google.com/")
#Página a la que queremos acceder

#Verificamos si el elemento con ID="nName" ya está cargado, este elemento es la caja de texto donde se hacen las busquedas
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "lst-ib")))
    #Obtenemos la caja de texto de busquedas
input_nombre = driver.find_element_by_id("lst-ib")
#Enviamos la cadena que estamos buscando
input_nombre.send_keys(cadena)
#Obtenemos el botón que ejecuta la búsqueda
boton = driver.find_element_by_name("btnK")
boton.click()   

WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.ID, "search")))
resultados = driver.find_element_by_id("search")
general = resultados.find_elements_by_class_name("g")
nombre = []
link = []
x=[]
for resultado in general:
    try:
        x=resultado.find_element_by_class_name('r')
        nombre.append(x.find_element_by_tag_name('a').get_attribute("text"))
        link.append(x.find_element_by_tag_name('a').get_attribute("href"))
    except NoSuchElementException:
        pass
for i in range(num_paginas):
    bnext = driver.find_element_by_id('pnnext')
    bnext.click()

    resultados = driver.find_element_by_id("search")
    general = resultados.find_elements_by_class_name("g")
    for resultado in general:
        try:
            x=resultado.find_element_by_class_name('r')
            nombre.append(x.find_element_by_tag_name('a').get_attribute("text"))
            link.append(x.find_element_by_tag_name('a').get_attribute("href"))
        except NoSuchElementException:
            pass
MD = pd.DataFrame({
    "Título": nombre,
    "url": link
    })
driver.close()
print('culminó')

culminó


In [267]:
data = MD
data

,Título,url
0,Facebook: Mauricio Diez Canseco presenta a sus...,https://larepublica.pe/espectaculos/1256281-fa...
1,Instagram: ella es la primera esposa de Mauric...,https://larepublica.pe/espectaculos/1247333-in...
2,Hija de Mauricio Diez Canseco remece Instagram...,https://larepublica.pe/espectaculos/1167353-hi...
3,Instagram: Hija de Mauricio Diez Canseco posa ...,https://larepublica.pe/espectaculos/1238569-in...
4,Instagram: hija de Mauricio Diez Canseco cauti...,https://larepublica.pe/espectaculos/1149940-in...
5,Instagram: Hija de Mauricio Diez Canseco prese...,https://larepublica.pe/espectaculos/1270648-in...
6,Instagram: Mauricio Diez Canseco posa junto a ...,https://larepublica.pe/tendencias/1214488-inst...
7,Facebook: esta fue la tierna sesión de fotos d...,https://larepublica.pe/espectaculos/1212080-fa...
8,Instagram: hija de Mauricio Diez Canseco posa ...,https://larepublica.pe/espectaculos/1281044-in...
9,Mauricio Diez Canseco renuncia a candidatura a...,https://larepublica.pe/espectaculos/703917-mau...


In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
import time

mainpath = "D:/Archivos Jordan Rodriguez/Retos/11.SrMD/04.Scraping/"
filename = "r_larepublico"
fullpath = os.path.join(mainpath,filename)

titulo = []
resumen = []
fecha = []
noticia = []
proxies = {"https": "https://pxsis.sunat.peru:8080"}
for i in range(len(data)):
    url = data["url"][i]
    y = None
    while not y:
        try:
            req = requests.get(url, proxies = proxies, verify = False)
            y = 5
        except:
            time.sleep(5)
            print('i')
    status_code = req.status_code
    if status_code == 200:
        print(i)
        html = BeautifulSoup(req.text, "html.parser")
        header=html.find('div',class_="post-internal_head")
        try:
            titulo.append(header.find('h1',class_="post-internal_title").getText())
            resumen.append(header.find('div',class_="post-internal_text").getText())
        except:
            titulo.append('No encontrado')
            resumen.append('No encontrado')
        #cabecera = header.find('div',class_="post-internal_head")
        try:
            fecha.append(header.find('p',class_="post-internal_date").getText())
        except:
            fecha.append('No encontrado')

        #cfecha=html.find('div',class_="post-internal_list-social atm_List-social")
        #fecha = cfecha.find('p')
        #cuerpo del articulo
        try:
            contenido=html.find('div',class_="col-16-md col-16-sm PostInternalBlockLeft__right")
            texto = contenido.find_all('p')
        except:
            text = 'No encontrado'
        index = []
        
        for i,entrada in enumerate(texto):
            index.append(entrada.getText())
        noticia.append(" ".join(index))
    else:
        print("break")
        time.sleep(10)
        pass
                
MD_2 = pd.DataFrame({
    "Título": titulo,
    "Resumen": resumen,
    "Fecha": fecha,
    "Contenido": noticia
    })

MD_2.to_excel(fullpath + "365a_.xlsx")
        

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


0


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


1


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


60


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


61


C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [259]:
import time
time.sleep(5)